# Super Mario Bros AI

Super Mario Bros AI Model built in Python based on [this tutorial](https://youtu.be/2eeYqJ0uBKE)

## Setup Super Mario

First setup the Mario Environment

#### Dependencies

In [1]:
!pip install gym_super_mario_bros nes_py

In [2]:
from nes_py.wrappers import JoypadSpace  # Import Joypad Wrapper
import gym_super_mario_bros  # Import game
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT # Import simplified controls

It is always important to simplify controls as much as possible when making an AI for a game. See list of controls provided by `SIMPLE_MOVEMENT` below

In [3]:
SIMPLE_MOVEMENT

[['NOOP'],
 ['right'],
 ['right', 'A'],
 ['right', 'B'],
 ['right', 'A', 'B'],
 ['A'],
 ['left']]

Now, setup `Super Mario` environment

In [4]:
# Setup game
env = gym_super_mario_bros.make('SuperMarioBros-v0')  # Check docs for other versions
env = JoypadSpace(env, SIMPLE_MOVEMENT)

# env.action_space now only has 7 possible actions (the 7 simplified moves)
# env.observation_space shows what's going on in the screen (in our case 240x256 pixels x 3 frames)

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/gym/envs/registration.py:564: UserWarning: WARN: The environment SuperMarioBros-v0 is out of date. You should consider upgrading to version `v3`.
  logger.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:97: UserWarning: WARN: We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html
  logger.warn(


In [5]:
# Create a flag to know when to restart
done = True
for step in range(1000):
    if done:
        env.reset()
    # Does random actions
    state, reward, done, info = env.step(env.action_space.sample())
    env.render()  # Show game

env.close()  # Closes at game
print("Done")

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:297: UserWarning: WARN: No render modes was declared in the environment (env.metadata['render_modes'] is None or not defined), you may have trouble when calling `.render()`
  logger.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:305: UserWarning: WARN: No render fps was declared in the environment (env.metadata['render_fps'] is None or not defined), rendering may occur at inconsistent fps
  logger.warn(
2022-07-26 19:49:10.618 Python[3956:166498] ApplePersistenceIgnoreState: Existing state will not be touched. New state will be written to /var/folders/f1/hs7h_prd4hz4drcqrr_jv0xh0000gn/T/org.python.python.savedState
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/gym_super_mario_bros/smb_env.py:148: RuntimeWarning: overflow encountered in ubyte_scalars
  return (self.ram[0x86] - 

Done


## Preprocessing

In this stage, we will *grayscale* the frames. Colored images carry 3 times as much data, which will take too long to process. We will also do *frame stacking*, which kind of gives the AI **memory** (it can see stuff like velocity of enemies, etc...)

In [ ]:
# Import FrameStack Wrapper and Grayscale Wrapper
from gym.wrappers import FrameStack, GrayScaleObservation

# Import Vectorization Wrappers
from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv

# Import matplotlib for visualization
from matplotlib import pyplot as plt

